In [1]:
from shor import QFT_d, QuantumCircuit, primes
from qiskit import transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
from fractions import Fraction
import numpy as np

In [7]:
gcd = np.gcd

In [8]:
backend = AerSimulator()

In [9]:
def c_amod15(a, power):
  if a not in [2,4,7,8,11,13]:
    raise ValueError("'a' must be 2,4,7,8,11 or 13")
  U = QuantumCircuit(4)
  for _ in range(power):
    if a in [2,13]:
      U.swap(2,3)
      U.swap(1,2)
      U.swap(0,1)
    if a in [7,8]:
      U.swap(0,1)
      U.swap(1,2)
      U.swap(2,3)
    if a in [4, 11]:
      U.swap(1,3)
      U.swap(0,2)
    if a in [7,11,13]:
      for q in range(4):
        U.x(q)
  U = U.to_gate()
  U.name = f"{a}^{power} mod 15"
  c_U = U.control()

  return c_U

In [16]:
# demo 1 circuit
n_count = 8
a = 7
qc = QuantumCircuit(n_count + 4, n_count)
qc.h(range(n_count))
qc.x(3+n_count)
for q in range(n_count):
  qc.append(c_amod15(a, 2**q), [q] + [i+n_count for i in range(4)])
qc.append(QFT_d(n_count), range(n_count))
qc.measure(range(n_count), range(n_count))
qc.draw(fold=-1)

┌───┐                                                                                                                             ┌───────┐┌─┐                     
 q_0: ┤ H ├───────■─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤0      ├┤M├─────────────────────
      ├───┤       │                                                                                                                     │       │└╥┘┌─┐                  
 q_1: ┤ H ├───────┼──────────────■──────────────────────────────────────────────────────────────────────────────────────────────────────┤1      ├─╫─┤M├──────────────────
      ├───┤       │              │                                                                                                      │       │ ║ └╥┘┌─┐               
 q_2: ┤ H ├───────┼──────────────┼──────────────■───────────────────────────────────────────────────────────────────────────────────────┤2      ├─╫──╫─┤M├───────────────
      ├───┤       │              │              │                                                                                       │       │ ║  ║ └╥┘┌─┐            
 q_3: ┤ H ├───────┼──────────────┼──────────────┼──────────────■────────────────────────────────────────────────────────────────────────┤3      ├─╫──╫──╫─┤M├────────────
      ├───┤       │              │              │              │                                                                        │  QFT† │ ║  ║  ║ └╥┘┌─┐         
 q_4: ┤ H ├───────┼──────────────┼──────────────┼──────────────┼──────────────■─────────────────────────────────────────────────────────┤4      ├─╫──╫──╫──╫─┤M├─────────
      ├───┤       │              │              │              │              │                                                         │       │ ║  ║  ║  ║ └╥┘┌─┐      
 q_5: ┤ H ├───────┼──────────────┼──────────────┼──────────────┼──────────────┼───────────────■─────────────────────────────────────────┤5      ├─╫──╫──╫──╫──╫─┤M├──────
      ├───┤       │              │              │              │              │               │                                         │       │ ║  ║  ║  ║  ║ └╥┘┌─┐   
 q_6: ┤ H ├───────┼──────────────┼──────────────┼──────────────┼──────────────┼───────────────┼───────────────■─────────────────────────┤6      ├─╫──╫──╫──╫──╫──╫─┤M├───
      ├───┤       │              │              │              │              │               │               │                         │       │ ║  ║  ║  ║  ║  ║ └╥┘┌─┐
 q_7: ┤ H ├───────┼──────────────┼──────────────┼──────────────┼──────────────┼───────────────┼───────────────┼────────────────■────────┤7      ├─╫──╫──╫──╫──╫──╫──╫─┤M├
      └───┘┌──────┴──────┐┌──────┴──────┐┌──────┴──────┐┌──────┴──────┐┌──────┴───────┐┌──────┴───────┐┌──────┴───────┐┌───────┴───────┐└───────┘ ║  ║  ║  ║  ║  ║  ║ └╥┘
 q_8: ─────┤0            ├┤0            ├┤0            ├┤0            ├┤0             ├┤0             ├┤0             ├┤0              ├──────────╫──╫──╫──╫──╫──╫──╫──╫─
           │             ││             ││             ││             ││              ││              ││              ││               │          ║  ║  ║  ║  ║  ║  ║  ║ 
 q_9: ─────┤1            ├┤1            ├┤1            ├┤1            ├┤1             ├┤1             ├┤1             ├┤1              ├──────────╫──╫──╫──╫──╫──╫──╫──╫─
           │  7^1 mod 15 ││  7^2 mod 15 ││  7^4 mod 15 ││  7^8 mod 15 ││  7^16 mod 15 ││  7^32 mod 15 ││  7^64 mod 15 ││  7^128 mod 15 │          ║  ║  ║  ║  ║  ║  ║  ║ 
q_10: ─────┤2            ├┤2            ├┤2            ├┤2            ├┤2             ├┤2             ├┤2             ├┤2              ├──────────╫──╫──╫──╫──╫──╫──╫──╫─
      ┌───┐│             ││             ││             ││             ││              ││              ││              ││               │          ║  ║  ║  ║  ║  ║  ║  ║ 
q_11: ┤ X ├┤3            ├┤3            ├┤3            ├┤3            ├┤3             ├┤3       

In [10]:
def qpe_amod15(a):
  N_COUNT = 8
  qc = QuantumCircuit(4+N_COUNT, N_COUNT)
  for q in range(N_COUNT):
    qc.h(q)     # Initialize counting qubits in state |+>
  qc.x(3+N_COUNT) # And auxiliary register in state |1>
  for q in range(N_COUNT): # Do controlled-U operations
    qc.append(
      c_amod15(a, 2**q),
      [q] + [i+N_COUNT for i in range(4)]
    )
  qc.append(QFT_d(N_COUNT), range(N_COUNT)) # Do inverse-QFT
  qc.measure(range(N_COUNT), range(N_COUNT))

  job = backend.run(transpile(qc, backend), shots=1, memory=True)
  readings = job.result().get_memory()
  print("Register: " + readings[0])
  phase = int(readings[0],2)/(2**N_COUNT)
  print(f"Phase: {phase}")
  return phase

In [11]:
N = 15
a = 7
phase = qpe_amod15(a) # Phase = s/r
frac = Fraction(phase).limit_denominator(15)
r = frac.denominator
guesses = [gcd(a**(r//2)-1, N), gcd(a**(r//2)+1, N)]
print(guesses)

Register Reading: 11000000
Corresponding Phase: 0.75
[3, 5]


In [13]:
FACTOR_FOUND = False
ATTEMPT = 0
while not FACTOR_FOUND:
  ATTEMPT += 1
  print(f"\nATTEMPT {ATTEMPT}:")
  phase = qpe_amod15(a) # Phase = s/r
  frac = Fraction(phase).limit_denominator(N)
  r = frac.denominator
  if r == 1:
    continue
  print(f"Result: r = {r}")
  if phase != 0:
    # Guesses for factors are gcd(x^{r/2} ±1 , 15)
    guesses = [gcd(a**(r//2)-1, N), gcd(a**(r//2)+1, N)]
    print(f"Guessed Factors: {guesses[0]} and {guesses[1]}")
    for guess in guesses:
      if guess not in [1,N] and (N % guess) == 0:
        # Guess is a factor!
        print(f"*** Non-trivial factor found: {guess} ***")
        FACTOR_FOUND = True


ATTEMPT 1:
Register Reading: 10000000
Corresponding Phase: 0.5
Result: r = 2
Guessed Factors: 3 and 1
*** Non-trivial factor found: 3 ***
